In [1]:
# !pip install tensorflow
!pip install pyarabic
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import pyarabic.araby as araby
import nltk
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import FastText
import time
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM




In [3]:
#downloading arabic stop words to remove from text
nltk.download('punkt')
nltk.download('stopwords')
arab_words=stopwords.words('arabic')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
data = pd.read_csv('/content/nlp.csv')


In [5]:
data= data.sample(n=10000,replace=True)
#radomly chose rows as it took long to process 400k rows:)


**DATA PREPROCESSING**

In [6]:
#dropping nulls
data=data.dropna()


In [7]:
 #created empty columns to observe if the change was made with each preprocessing technique
data["codes"] = ""
data["normalized"] = ""
data["diacritics"] = ""
data["stop_words"] = ""

data.columns

Index(['id', 'dialect', 'transformed', 'fineText', 'codes', 'normalized',
       'diacritics', 'stop_words'],
      dtype='object')

In [8]:
data['fineText']=data['fineText'].astype(str)

In [9]:
#remove escsape codes
data['code'] = data['fineText'].apply(lambda x:x.strip("\n"))
data['code'] = data['fineText'].apply(lambda x:x.strip("\t"))
data['code'] = data['fineText'].apply(lambda x:x.strip("\\"))
data['code'] = data['fineText'].apply(lambda x:x.strip())


In [10]:
#normalize words
data['normalized'] = data['code'].apply(lambda x:re.sub("[إأٱآا]", "ا", x))
data['normalized'] = data['code'].apply(lambda x:re.sub( "ي","ى", x))
data['normalized'] = data['code'].apply(lambda x:re.sub("ة", "ه", x))


In [11]:
#remove tashkeel
data['diacritics'] = data['normalized'].apply(lambda x:araby.strip_tashkeel(x))



In [12]:
# data['diacritics'] = data['diacritics'].apply(word_tokenize)


In [13]:
# remove stop words
#stop words is the final column after all the changes, i did it for you to observe the change but then i noticed the data is already preprocessed:)
data['stop_words'] = data['diacritics'].apply(lambda x:[word for word in x if word not in arab_words])


In [14]:
#converting the finetext to separate independent words
data['fineText'] = data['fineText'].apply(lambda x: [str(word.split()) for word in x.split()]) 
data['fineText'] = data['fineText'].apply(lambda x: ' '.join(map(str, x)))
data

,id,dialect,transformed,fineText,codes,normalized,diacritics,stop_words,code
454034,114802931177504768,BH,17,['هذى'] ['صوره'] ['الجنازه'] ['شكلها'] ['واحد'...,,هذى صوره الجنازه شكلها واحد بس,هذى صوره الجنازه شكلها واحد بس,"[ , , , , ]",هذى صوره الجنازه شكلها واحد بس
61055,1065594219220410368,QA,2,['عندك'] ['مانع'] ['لاسمح'] ['الله'],,عندك مانع لاسمح الله,عندك مانع لاسمح الله,"[ , , ]",عندك مانع لاسمح الله
366894,1124966543882895360,KW,13,['كاك'] ['لا'] ['صج'] ['شصاير'] ['بالدنيا'],,كاك لا صج شصاير بالدنيا,كاك لا صج شصاير بالدنيا,"[ , , , ]",كاك لا صج شصاير بالدنيا
217633,696033302889639936,SA,8,['ماتت'] ['الحماره'] ['وعقبال'] ['الجحش'],,ماتت الحماره وعقبال الجحش,ماتت الحماره وعقبال الجحش,"[ , , ]",ماتت الحماره وعقبال الجحش
416298,979069686708166784,AE,16,['ربى'] ['يحفظج'] ['من'] ['العين'],,ربى يحفظج من العين,ربى يحفظج من العين,"[ , , ]",ربى يحفظج من العين
...,...,...,...,...,...,...,...,...,...
123643,954506738379689984,PL,3,['وراك'] ['وراك'] ['والمافيا'] ['طول'] ['معاك'],,وراك وراك والمافيا طول معاك,وراك وراك والمافيا طول معاك,"[ , , , ]",وراك وراك والمافيا طول معاك
147300,1118230775554506752,TN,5,['الغاها'] ['الكلب'] ['بلننى'] ['صحيح'] ['يا']...,,الغاها الكلب بلننى صحيح يا كلب,الغاها الكلب بلننى صحيح يا كلب,"[ , , , , ]",الغاها الكلب بلننى صحيح يا كلب
21577,1169961161779027968,LY,1,['مش'] ['بطريقه'] ['هذى'] ['يتغيروا'] ['فى'] [...,,مش بطريقه هذى يتغيروا فى حلولوا اخرى,مش بطريقه هذى يتغيروا فى حلولوا اخرى,"[ , , , , , ]",مش بطريقه هذى يتغيروا فى حلولوا اخرى
353966,1000376594467360768,KW,13,['الله'] ['يجمل'] ['عيونك'] ['ويديم'] ['سعادتك'],,الله يجمل عيونك ويديم سعادتك,الله يجمل عيونك ويديم سعادتك,"[ , , , ]",الله يجمل عيونك ويديم سعادتك


- NAIVE BAYES CLASSIFER

In [15]:
# Encode the categories into numerical values
le = LabelEncoder()
data['dialect_encode'] = le.fit_transform(data['dialect'])


In [16]:
#this will be used to divide data into train and test
X = data['fineText'].astype(str)
y = data['dialect_encode']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34, stratify=y)

In [18]:
#convert the data into numbers to fit the naive bayes algorithm ill use below
# is used to convert a collection of text documents to a matrix of TF-IDF features.
vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(X_train)

In [19]:
tfidf_test = vectorizer.transform(X_test)


In [20]:
#applying the naive bayes model on my vectorized data and the labels 
#da el naive bayes ely monaseb lel text classification
nb = MultinomialNB()
# Fit the model
nb.fit(tfidf_train, y_train)
#fit the training data to the model el label bel sentences

MultinomialNB()

In [21]:
print("Accuracy:",nb.score(tfidf_test, y_test))


Accuracy: 0.245


In [22]:
from sklearn.naive_bayes import MultinomialNB
nb2 = MultinomialNB(alpha=0.6)
# Fit the model
nb2.fit(tfidf_train, y_train)

MultinomialNB(alpha=0.6)

In [23]:
print("Accuracy:",nb2.score(tfidf_test, y_test))
# This  prints the accuracy of the MultinomialNB model on the testing data using the score method.


Accuracy: 0.277


In [24]:
from sklearn import naive_bayes
predictions = nb.predict(tfidf_test)
predictions
# uses the predict method of the MultinomialNB instance to make predictions on the testing data, and stores the predictions in the predictions variable.

array([11,  6,  3, ...,  3,  3,  3])

In [25]:
#f score of 1st model
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average='weighted')

0.16072266314338268

In [26]:
from sklearn import naive_bayes
predictions2 = nb2.predict(tfidf_test)
predictions2
#new unseen data

array([11,  6,  3, ...,  3,  3,  3])

In [27]:
#f score of second model
from sklearn.metrics import f1_score
f1_score(y_test, predictions2, average='weighted')

0.19745087616380508

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

#  predictions on the test data
predictions = nb.predict(tfidf_test)
# Generate a classification report and confusion matrix
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      0.03      0.05       118
           1       0.00      0.00      0.00       113
           2       1.00      0.01      0.03        73
           3       0.20      0.98      0.33       244
           4       0.00      0.00      0.00        69
           5       0.00      0.00      0.00       124
           6       0.23      0.52      0.32       193
           7       0.69      0.09      0.16       118
           8       0.69      0.26      0.38       157
           9       0.00      0.00      0.00        52
          10       0.00      0.00      0.00        81
          11       0.29      0.41      0.34       190
          12       0.55      0.11      0.19       140
          13       0.00      0.00      0.00       114
          14       0.00      0.00      0.00        63
          15       0.00      0.00      0.00        67
          16       0.00      0.00      0.00        43
          17       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
from sklearn.metrics import classification_report, confusion_matrix

#  predictions on the test data
predictions = nb2.predict(tfidf_test)

# Generate a classification report and confusion matrix
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.67      0.05      0.09       118
           1       0.33      0.01      0.02       113
           2       1.00      0.03      0.05        73
           3       0.23      0.95      0.37       244
           4       0.00      0.00      0.00        69
           5       0.38      0.02      0.05       124
           6       0.24      0.59      0.35       193
           7       0.81      0.19      0.30       118
           8       0.60      0.34      0.43       157
           9       0.00      0.00      0.00        52
          10       1.00      0.02      0.05        81
          11       0.28      0.48      0.35       190
          12       0.46      0.19      0.26       140
          13       0.50      0.02      0.03       114
          14       0.00      0.00      0.00        63
          15       0.00      0.00      0.00        67
          16       0.00      0.00      0.00        43
          17       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM

In [30]:
#VECTORIZING the data as above to be represented as numbers
X = data['fineText'] 
y = data['dialect']
vectorizer = TfidfVectorizer()
X_new = vectorizer.fit_transform(X)

In [31]:
# DIVIDING TO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(X_new, data['dialect_encode'], test_size=0.2, random_state=1)


In [32]:
#CONVERTING THE X TRAIN TO ARRAY
newXTrain=X_train.toarray()


In [33]:
# CONVERT THE X TEST TO ARRAY
newXTest=X_test.toarray()

In [34]:
finalXtrain=np.asarray(newXTrain).astype('float32').reshape(newXTrain.shape[0],1,newXTrain.shape[1])
finalXtest=np.asarray(newXTest).astype('float32').reshape(newXTest.shape[0],1,newXTest.shape[1])

In [35]:
#CHANGING THE LABELS TO VECTORS THAT PUT 1 IN PLACE OF THE CLASS FOR EXAMPLE CLASS 2 WILL BE [0,1,0,0]
from keras.utils import to_categorical
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)


In [36]:
# initializing the lstm model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(18, activation='softmax'))

In [37]:
# Train the LSTM model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(finalXtrain, y_train_hot, epochs=10, batch_size=64, validation_data=(finalXtest, y_test_hot))



Epoch 1/10
125/125 [==============================] - 49s 328ms/step - loss: 2.8213 - accuracy: 0.1293 - val_loss: 2.7367 - val_accuracy: 0.1260
Epoch 2/10
125/125 [==============================] - 34s 275ms/step - loss: 2.5698 - accuracy: 0.1831 - val_loss: 2.5990 - val_accuracy: 0.2230
Epoch 3/10
125/125 [==============================] - 36s 290ms/step - loss: 2.0444 - accuracy: 0.5107 - val_loss: 2.3576 - val_accuracy: 0.3240
Epoch 4/10
125/125 [==============================] - 33s 265ms/step - loss: 1.2110 - accuracy: 0.8611 - val_loss: 2.1642 - val_accuracy: 0.3750
Epoch 5/10
125/125 [==============================] - 33s 260ms/step - loss: 0.5330 - accuracy: 0.9674 - val_loss: 2.0930 - val_accuracy: 0.3870
Epoch 6/10
125/125 [==============================] - 34s 268ms/step - loss: 0.2224 - accuracy: 0.9900 - val_loss: 2.0905 - val_accuracy: 0.3825
Epoch 7/10
125/125 [==============================] - 34s 275ms/step - loss: 0.1069 - accuracy: 0.9964 - val_loss: 2.0973 - val_ac

In [38]:
# Evaluate the LSTM model
loss, acc = model.evaluate(finalXtest, y_test_hot)
print("Accuracy: {:.2f}%".format(acc * 100))

63/63 [==============================] - 2s 29ms/step - loss: 2.1415 - accuracy: 0.3785
Accuracy: 37.85%


In [39]:
#classification report, i wasnt able to observe the f scores of the best classified classes as there was a problem with the ram, but im sure there will be an output
y_pred = model.predict(finalXtest)
y_pred = np.argmax(y_pred, axis=1)
from sklearn import metrics
y_pred=np.around(model.predict(finalXtest))
print(metrics.classification_report(y_test_hot,y_pred))

63/63 [==============================] - 2s 30ms/step
              precision    recall  f1-score   support

           0       0.73      0.12      0.21       133
           1       0.35      0.08      0.13       112
           2       0.87      0.20      0.33        64
           3       0.78      0.54      0.64       251
           4       0.60      0.04      0.08        69
           5       0.47      0.13      0.21       137
           6       0.52      0.21      0.30       189
           7       0.79      0.31      0.45       121
           8       0.80      0.35      0.48       135
           9       0.80      0.09      0.16        46
          10       0.38      0.04      0.07        81
          11       0.46      0.23      0.31       184
          12       0.41      0.14      0.21       134
          13       0.57      0.10      0.18       125
          14       0.62      0.10      0.17        52
          15       0.71      0.06      0.11        81
          16       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
#note i wasnt able to run the last cells cause of the ram, but iam sure it performed better than naive eo please re run this part maybe it wont lag in your laptop 

In [41]:
model2 = Sequential()
model2.add(LSTM(256))
model2.add(Dense(18, activation='softmax'))

In [42]:
# Train the LSTM model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(finalXtrain, y_train_hot, epochs=10, batch_size=64, validation_data=(finalXtest, y_test_hot))



Epoch 1/10
125/125 [==============================] - 66s 502ms/step - loss: 2.7972 - accuracy: 0.1330 - val_loss: 2.7111 - val_accuracy: 0.1260
Epoch 2/10
125/125 [==============================] - 62s 493ms/step - loss: 2.4459 - accuracy: 0.2754 - val_loss: 2.4795 - val_accuracy: 0.2850
Epoch 3/10
125/125 [==============================] - 70s 562ms/step - loss: 1.6122 - accuracy: 0.7148 - val_loss: 2.1771 - val_accuracy: 0.3690
Epoch 4/10
125/125 [==============================] - 62s 499ms/step - loss: 0.6502 - accuracy: 0.9511 - val_loss: 2.0631 - val_accuracy: 0.3935
Epoch 5/10
125/125 [==============================] - 62s 493ms/step - loss: 0.2047 - accuracy: 0.9891 - val_loss: 2.0618 - val_accuracy: 0.3900
Epoch 6/10
125/125 [==============================] - 65s 521ms/step - loss: 0.0812 - accuracy: 0.9970 - val_loss: 2.0758 - val_accuracy: 0.3845
Epoch 7/10
125/125 [==============================] - 66s 527ms/step - loss: 0.0416 - accuracy: 0.9991 - val_loss: 2.0973 - val_ac

In [43]:
# Evaluate the LSTM model
loss, acc = model2.evaluate(finalXtest, y_test_hot)
print("Accuracy: {:.2f}%".format(acc * 100))

63/63 [==============================] - 4s 65ms/step - loss: 2.1557 - accuracy: 0.3750
Accuracy: 37.50%
